# MSTX Summary

集群场景MSTX打点数据分析

主要包含以下2个统计内容：
1. 按Step分组的，整个集群MSTX打点数据的统计情况
2. 按Name分组的，每个Rank上MSTX打点数据的统计情况

### 数据准备

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import plotly.offline as pyo

def is_lab_notebook():
    import re
    import psutil
    return any(re.search('jupyter--lab-script', x) for x in psutil.Process().parent().cmdline())

if is_lab_notebook():
    pyo.init_notebook_mode()

import pandas as pd
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 1000)

import cluster_display

all_fwk_stats_gdf = pd.read_csv("all_fwk_stats.csv", index_col="Name").groupby("StepId")
all_cann_stats_gdf = pd.read_csv("all_cann_stats.csv", index_col="Name").groupby("StepId")
all_device_stats_gdf = pd.read_csv("all_device_stats.csv", index_col="Name").groupby("StepId")
mark_stats_df = pd.read_csv("mark_stats.csv", index_col="Name")

### 集群MSTX数据分析

将整个集群所有Rank的MSTX数据进行汇总，按Step划分，统计分析耗时情况，时间单位为毫秒(ms)
打点数据分为三种：
1. 框架侧耗时：Framework Time
2. Cann侧耗时：Cann Time
3. Device侧耗时：Devcie Time

3种数据都包含以下统计项：
- Count：数量
- Mean：平均耗时
- Std：标准差
- Min：最小值
- Q1：四分之一分位数
- Median：中位数
- Q3：四分之三分位数
- Max：最大值
- Sum：总耗时

In [ ]:
def display_stats_mstx_step_combobox(selected, args):
    step = selected
    fwk_stats_gdf, cann_stats_gdf, device_stats_gdf = args
    fwk_df = fwk_stats_gdf.get_group(step)
    cann_df = cann_stats_gdf.get_group(step)
    device_df = device_stats_gdf.get_group(step)
    figs = []
    display(HTML("<p><b>Framework Time Stats</b></p>"))
    display(fwk_df)
    cluster_display.display_duration_boxplots(figs, fwk_df, title="Framework Time", x_title="Name", y_title="Time")
    display(HTML("<p><b>Cann Time Stats</b></p>"))
    display(cann_df)
    cluster_display.display_duration_boxplots(figs, cann_df, title="Cann Time", x_title="Name", y_title="Time")
    display(HTML("<p><b>Device Time Stats</b></p>"))
    display(device_df)
    cluster_display.display_duration_boxplots(figs, device_df, title="Device Time", x_title="Name", y_title="Time")

steps = list(all_fwk_stats_gdf.groups.keys())
if steps:
    cluster_display.display_stats_optional_combobox(steps, display_stats_mstx_step_combobox, 
                                                    [all_fwk_stats_gdf, all_cann_stats_gdf, all_device_stats_gdf], "Step:")
else:
    print("There is no step in stats, so no need to display")

### 集群Rank MSTX数据分析

将集群内每个Rank的MSTX数据进行汇总，按打点Name分类，统计分析耗时情况，时间单位为毫秒(ms)

包含以下统计项：
- Name：打点名称
- FrameworkDuration(Ms)：框架侧耗时
- CannDuration(Ms)：Cann侧耗时
- DeviceDuration(Ms)：Device侧耗时
- Rank：Rank序号
- StepId：Step序号

In [ ]:
def display_mstx_duration_by_rank(selected, args):
    mark_stats_gdf = args
    df = mark_stats_gdf.get_group(selected).sort_values("Rank")
    display(df)
    fwk_duration = []
    cann_duration = []
    device_duration = []
    step_ids = []
    for step_id, step_df in df.groupby("StepId"):
        fwk_duration.append((step_id, step_df["FrameworkDuration(Ms)"].values))
        cann_duration.append((step_id, step_df["CannDuration(Ms)"].values))
        device_duration.append((step_id, step_df["DeviceDuration(Ms)"].values))
        step_ids.append(step_id)
    fwk_df = pd.concat([pd.Series(dur, name=step_id) for step_id, dur in fwk_duration], axis=1)
    cann_df = pd.concat([pd.Series(dur, name=step_id) for step_id, dur in cann_duration], axis=1)
    device_df = pd.concat([pd.Series(dur, name=step_id) for step_id, dur in device_duration], axis=1)
    figs = []
    ranks = df["Rank"].drop_duplicates()
    cluster_display.display_graph(figs, ranks, fwk_df[step_ids],
                                  title="Framework Time", x_title="Rank", y_title="Time", legend_title="Step")
    cluster_display.display_graph(figs, ranks, cann_df[step_ids],
                                  title="Cann Time", x_title="Rank", y_title="Time", legend_title="Step")
    cluster_display.display_graph(figs, ranks, device_df[step_ids],
                                  title="Device Time", x_title="Rank", y_title="Time", legend_title="Step")

mark_stats_gdf = mark_stats_df.groupby(mark_stats_df.index)
names = list(mark_stats_gdf.groups.keys())
if steps:
    cluster_display.display_stats_optional_combobox(names, display_mstx_duration_by_rank, mark_stats_gdf, "Name:")
else:
    print("There is no mark name in stats, so no need to display")